In [7]:
from deap import base, creator, tools
import numpy as np
import random
# base.Fitnessを継承するFitnessMaxクラスを定義する
# 最大化問題のためweightsは(1.0,)
# 注意：1.0のあとにカンマが必要
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
# listを継承するIndividualクラスを定義する
# GAの個体となる
creator.create('Individual', list, fitness=creator.FitnessMax)
# ツールボックスを定義する
toolbox = base.Toolbox()
# ツールボックスにattr_boolオペレータを登録する
# attr_boolオペレータは、0,1のランダムな変数を返す
toolbox.register('attr_bool', random.randint, 0,1)
# individualオペレータを登録する
# individualオペレータは、attr_boolオペレータを100回実行した結果をIndividualクラスに代入する
# 具体的にはランダムに0,1の値をもつ100要素からなるリストを定義している
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.attr_bool, 100)
# pupulationオペレータを登録する
# tools.initRepeatのn（何回実行するか）はオペレータ実行時に指定する
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
# individualはリスト（正確にはlistを継承したcreator.Individualクラス）
# 注意：評価値はタプルで返す必要があるため、sum()のあとに,が必要
def evalOneMax(individual):
    return sum(individual),
# evaluateオペレータを登録する、GAの評価関数
toolbox.register('evaluate', evalOneMax)
# mateオペレータを登録する、GAの交叉、tools.cxTwoPointは2点交叉
toolbox.register('mate', tools.cxTwoPoint)
# mutateオペレータを登録する、GAの突然変異、tools.mutFlipBitはランダムにビットを反転
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)
# selectオペレータを登録する、GAの選択、tools.selTournamentはトーナメント選択
toolbox.register('select', tools.selTournament, tournsize=3)
# 定数の定義
# 交叉率
CXPB = 0.5
# 突然変異率
MUTPB = 0.2
# 個体数300の遺伝子プールを作成する
pop = toolbox.population(n=300)
# 各個体に評価関数を適用し適応度を取得する
fitnesses = list(map(toolbox.evaluate, pop))
# 各個体に適応度を付与する
# zip関数でpopから1個体、fitnessesから1適応度をそれぞれ取得する
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit
# 結果表示用の変数を定義する
best_ind = None # 最良個体
worst_ind = None # 最悪個体
best_fits = None # 最良値
worst_fits = None # 最悪値
mean_fits = None # 適応度の平均値
std_fits = None # 適応度の標準偏差
print('-----ループ開始------')
# 世代数を0で初期化する
g = 0
# fitnessesはタプルなので、適応度の第1要素のみ取得
fits = [ind.fitness.values[0] for ind in pop]

# 個体のサイズが100なので、OneMax問題の最適解は100となる
# 適応度が100の個体がある場合はループを終了する
# また世代数が1000世代になった場合はループを終了する
while max(fits) < 100 and g < 1000:
    # 選択 offspringは子孫という意味
    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))
    # 交叉
    # offspringの偶数番[::2]とoffspringの奇数番[1::2]の個体を取り出し交叉する
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        # [0.0,1.0]の乱数を発生させ、交叉率（CXPB）未満のときに交叉を実施する
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            # 交叉した個体の評価値を削除する
            del child1.fitness.values
            del child2.fitness.values
    # 突然変異
    for mutant in offspring:
        # [0.0,1.0]の乱数を発生させ、突然変異率（MUTPB）未満のときに突然変異を実施する
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            # 突然変異した個体の評価値を削除する
            del mutant.fitness.values
    # 交叉または突然変異した個体のみ取得する（評価値を削除した個体のfitness.validはFalseとなっている）
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    # 再評価対象の個体を評価し適応度を付与する
    fitness = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitness):
        ind.fitness.values = fit
    # 遺伝子プールを更新する
    pop = offspring
    # fitnessesはタプルなので、適応度の第1要素のみ取得
    fits = [ind.fitness.values[0] for ind in pop]
    # 最良値、最悪値などの統計情報の計算
    best_ind = pop[np.argmax(fits)]
    worst_ind = pop[np.argmin(fits)]
    best_fits = np.max(fits)
    worst_fits = np.min(fits)
    mean_fits = np.mean(fits)
    std_fits = np.std(fits)
    # 世代数を1つ増やす
    g += 1
    # 10世代ごとにログを表示する
    if g == 1 or g % 10 == 0:
        print('世代数：{}'.format(g))
        print('最良個体：{}'.format(best_ind))
        print('最良値：{}'.format(best_fits))
        print('最悪個体：{}'.format(worst_ind))
        print('最悪値：{}'.format(worst_fits))
        print('平均値：{:.4f}'.format(mean_fits))
        print('標準偏差：{:.4f}'.format(std_fits))
        print('------------------')
print('-----ループ終了------')
print('世代数：{}'.format(g))
print('最良個体：{}'.format(best_ind))
print('最良値：{}'.format(best_fits))
print('最悪個体：{}'.format(worst_ind))
print('最悪値：{}'.format(worst_fits))
print('平均値：{:.4f}'.format(mean_fits))
print('標準偏差：{:.4f}'.format(std_fits))
print('------------------') 

-----ループ開始------
世代数：1
最良個体：[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0]
最良値：65.0
最悪個体：[1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]
最悪値：43.0
平均値：54.7233
標準偏差：3.8575
------------------
世代数：10
最良個体：[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 

In [4]:
!pip install deap

     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.1 MB 388.9 kB/s eta 0:00:03
     ------ --------------------------------- 0.2/1.1 MB 1.3 MB/s eta 0:00:01
     --------------------- ------------------ 0.6/1.1 MB 3.3 MB/s eta 0:00:01
     -------------------------------------- - 1.0/1.1 MB 4.6 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for deap: filename=deap-1.4.1-cp311-cp311-win_amd64.whl size=108750 sha256=2a1a0b836a69065855c9daa717c495c4a53a34c8f2b329c2bab18aa1ce72e004
  Stored in directory: c:\users\81808\appdata\local\pip\cache\wheels\f8\64\b8\65eacfbff3024ae2e2beb22e691d5c8abb89fbd863b8049b5f
Successfully built deap
